In [1]:
# # header
import sys
sys.path.append(r"../")

%load_ext autoreload
%autoreload 2

In [ ]:
# # built-in modules
import os
from pprint import pformat
from collections import OrderedDict
# # Torch modules
import torch
from torch.utils.data import random_split
from torchvision import transforms, datasets
# # internal imports
from prelude import startup_folders, get_device, load_dicts
from src.composer import Recognition_DS, PatternedRecognition_DS
from src.model import AttentionModel
from src.utils import plot_all
from src.utils import build_loaders
from src.conductor import AttentionTrain


In [3]:
start_folder = r"../pretrained/mnist_v2"
results_folder, logger = startup_folders(start_folder, name=f"exp_a")
data_path = r"../data"

../pretrained/mnist_v2/1753527155 was created!


In [4]:
model_params = load_dicts(start_folder, "model_params")
tasks = load_dicts(start_folder, "tasks")
train_params = load_dicts(start_folder, "train_params")
DeVice, num_workers, pin_memory = get_device()
print(f"model_params: {model_params}")
print(f"tasks: {tasks}")
print(f"train_params: {train_params}")
tasks['Recognition']

Device set to mps
model_params: {'in_dims': [3, 96, 96], 'n_classes': 10, 'out_dim': 20, 'normalize': True, 'softness': 0.5, 'channels': [3, 16, 32, 64, 128, 128], 'residuals': False, 'kernels': 3, 'strides': 1, 'paddings': 1, 'conv_bias': True, 'conv_norms': [None, 'layer', 'layer', 'layer', 'layer'], 'conv_dropouts': 0.0, 'conv_funs': ReLU(), 'deconv_funs': Tanh(), 'deconv_norms': [None, 'layer', 'layer', 'layer', 'layer'], 'pools': [2, 2, 2, 2, 3], 'rnn_dims': [128, 64], 'rnn_bias': True, 'rnn_dropouts': 0.0, 'rnn_funs': ReLU(), 'n_tasks': 7, 'task_layers': 1, 'task_weight': True, 'task_bias': True, 'task_funs': Tanh(), 'norm_mean': [0.5, 0.5, 0.5], 'norm_std': [1.0, 1.0, 1.0], 'rnn_to_fc': False, 'trans_fun': ReLU()}
tasks: {'IOR': {'composer': 'IOR_DS', 'key': 0, 'params': {'n_digits': 3, 'n_attend': 2, 'noise': 0.25, 'overlap': 1.0}, 'datasets': ['IOR_DS', 'IOR_DS', 'IOR_DS'], 'dataloaders': [None, None, None], 'loss_w': [1.0, 1.0, 0.0], 'loss_s': [None, None], 'has_prompt': Fals

{'composer': 'Recognition_DS',
 'key': 4,
 'params': {'n_iter': 3,
  'stride': 16,
  'blank': False,
  'static': False,
  'noise': 0.25},
 'datasets': ['Recognition_DS', 'Recognition_DS', 'Recognition_DS'],
 'dataloaders': [None, None, None],
 'loss_w': [1.0, 0.0, 2.0],
 'loss_s': '(slice(1, None, None), None)',
 'has_prompt': False}

In [6]:
tasks = OrderedDict({})

tasks["Rec_VisDyn"] = {
    "composer": Recognition_DS,
    "key": 4,
    "params": {"n_iter": 3, "stride": 16, "blank": False, "static": False, "noise": 0.25},
    "datasets": [],
    "dataloaders": [],
    "loss_w": (1.0, 0.0, 2.0),  # labels, masks, last label
    "loss_s": (slice(1, None), None),  # labels, masks
    "has_prompt": False,
}
tasks["Rec_VisStat"] = {
    "composer": Recognition_DS,
    "key": 4,
    "params": {"n_iter": 3, "stride": 16, "blank": False, "static": True, "noise": 0.25},
    "datasets": [],
    "dataloaders": [],
    "loss_w": (1.0, 0.0, 2.0),  # labels, masks, last label
    "loss_s": (slice(1, None), None),  # labels, masks
    "has_prompt": False,
}
tasks["Rec_inVisDyn"] = {
    "composer": Recognition_DS,
    "key": 4,
    "params": {"n_iter": 3, "stride": 16, "blank": True, "static": False, "noise": 0.25},
    "datasets": [],
    "dataloaders": [],
    "loss_w": (1.0, 0.0, 2.0),  # labels, masks, last label
    "loss_s": (slice(1, None), None),  # labels, masks
    "has_prompt": False,
}
tasks["Rec_inVisStat"] = {
    "composer": Recognition_DS,
    "key": 4,
    "params": {"n_iter": 3, "stride": 16, "blank": True, "static": True, "noise": 0.25},
    "datasets": [],
    "dataloaders": [],
    "loss_w": (1.0, 0.0, 2.0),  # labels, masks, last label
    "loss_s": (slice(1, None), None),  # labels, masks
    "has_prompt": False,
}

In [7]:
tasks["PatternedRecognition"] = {
    "composer": PatternedRecognition_DS,
    "key": 4,
    "params": {"n_iter": 3, "noise": 0.25},
    "datasets": [],
    "dataloaders": [],
    "loss_w": (0.0, 0.0, 0.0),  # labels, masks, last label
    "loss_s": (None, None),  # labels, masks
    "has_prompt": False,
}

In [8]:
# datasets and dataloaders
tralid_ds = datasets.MNIST(root=data_path, train=True, download=True, transform=transforms.ToTensor())
test_ds = datasets.MNIST(root=data_path, train=False, download=True, transform=transforms.ToTensor())
train_ds, valid_ds = random_split(tralid_ds, (50000, 10000))
DeVice, num_workers, pin_memory = get_device()
for o in tasks:
    tasks[o]["datasets"].append(tasks[o]["composer"](train_ds, **tasks[o]["params"]))
    tasks[o]["datasets"].append(tasks[o]["composer"](valid_ds, **tasks[o]["params"]))
    tasks[o]["datasets"].append(tasks[o]["composer"](test_ds, **tasks[o]["params"]))
    tasks[o]["datasets"][1].build_valid_test()
    tasks[o]["datasets"][2].build_valid_test()
    tasks[o]["dataloaders"] = build_loaders(tasks[o]["datasets"], batch_size=train_params["batch_size"], num_workers=num_workers, pin_memory=pin_memory)

Device set to mps


In [9]:
# create a blank model
model = AttentionModel(**model_params)
conductor = AttentionTrain(model, None, None, tasks, logger, results_folder)

# load states into the model
model_dir = os.path.join(start_folder, "model" + ".pth")
assert os.path.exists(model_dir), "Could not find the model.pth in the given dir!"
model.load_state_dict(torch.load(model_dir, map_location=DeVice))

<All keys matched successfully>

In [ ]:
# evaluating...
conductor.eval(DeVice, "test", False)
# plotting...
plot_all(10, model, tasks, results_folder, "_test", DeVice, logger, False, "test")

testing...
  Task Rec_VisDyn:
    CEi Loss: 0.075    CEe Loss: 0.032    Pix Err: 0.013    Att Acc: 0.853    Cls Acc: 9910/10000
  Task Rec_VisStat:
    CEi Loss: 0.182    CEe Loss: 0.095    Pix Err: 0.014    Att Acc: 0.856    Cls Acc: 9691/10000
  Task Rec_inVisDyn:
    CEi Loss: 0.400    CEe Loss: 0.159    Pix Err: 0.015    Att Acc: 0.788    Cls Acc: 9517/10000
  Task Rec_inVisStat:
    CEi Loss: 1.157    CEe Loss: 0.642    Pix Err: 0.015    Att Acc: 0.779    Cls Acc: 8400/10000
  Task PatternedRecognition:
    CEi Loss: 2.365    CEe Loss: 0.084    Pix Err: 0.014    Att Acc: 0.829    Cls Acc: 9716/10000


In [12]:
# evaluating sequence one iteration at a time
conductor.eval_seq(DeVice, "test", None)

testing...
  Task Rec_VisDyn:
    CEi Loss: 2.150    Pix Err: 0.021    Att Acc: 0.926    Cls Acc: 2786/10000
    CEi Loss: 0.047    Pix Err: 0.010    Att Acc: 0.692    Cls Acc: 9862/10000
    CEi Loss: 0.035    Pix Err: 0.009    Att Acc: 0.941    Cls Acc: 9899/10000
    CEi Loss: 0.036    Pix Err: 0.009    Att Acc: 0.769    Cls Acc: 9900/10000
  Task Rec_VisStat:
    CEi Loss: 2.147    Pix Err: 0.021    Att Acc: 0.926    Cls Acc: 2844/10000
    CEi Loss: 0.086    Pix Err: 0.010    Att Acc: 0.698    Cls Acc: 9725/10000
    CEi Loss: 0.091    Pix Err: 0.010    Att Acc: 0.948    Cls Acc: 9721/10000
    CEi Loss: 0.092    Pix Err: 0.010    Att Acc: 0.753    Cls Acc: 9725/10000
  Task Rec_inVisDyn:
    CEi Loss: 1.853    Pix Err: 0.024    Att Acc: 0.841    Cls Acc: 5062/10000
    CEi Loss: 0.263    Pix Err: 0.011    Att Acc: 0.632    Cls Acc: 9186/10000
    CEi Loss: 0.169    Pix Err: 0.009    Att Acc: 0.890    Cls Acc: 9500/10000
    CEi Loss: 0.173    Pix Err: 0.011    Att Acc: 0.671    C